In [3]:
# import packages for data analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
# import glob to load .csv data files
import glob
import itertools

ModuleNotFoundError: No module named 'pandas'

In [2]:
## Create DataFrame by concatenating 'make' used car data files

df = pd.DataFrame()
def get_data():
    for file_name in glob.glob('*.csv'):

        if 'unclean' not in file_name and 'cclass' not in file_name and 'focus' not in file_name:
            pulled = pd.read_csv(file_name, low_memory=False)
            pulled['make'] = file_name[:-4]

            try:
                pulled.rename(columns = {'tax(£)':'tax'}, inplace = True)
            except:
                continue
            df = pd.concat([df,pulled],axis=0)
    
    return df

df = get_data()

NameError: name 'pd' is not defined

In [ ]:
# Create dataframe for categorical columns 

def create_dummy_df(df, cat_cols):

    for col in cat_cols:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True)], axis=1)
        except:
            continue
    return df.astype(np.float)

dum_df = create_dummy_df(df, cat_cols)

In [ ]:
# Recast year as age to make interpretation easier, and drop year

df['age'] = 2020 - df['year']
df.drop(['year'], axis=1, inplace=True)


In [ ]:
# Create heatmap for correlation matrix to study correlation between numerical columns

sns.heatmap(df.corr(), annot=True, fmt='.2f')
plt.show()

In [ ]:
# Extract catagorical columns to be recast a dummy columns

cat_df = df.select_dtypes(include=['object']).copy()
cat_cols = cat_df.columns

In [ ]:
# These lists contain groups of features to aid investigation

mod_list = [col for col in dum_df.columns if 'model' in col]
mil_list = ['mileage']
tax_list = ['tax']
mpg_list = ['mpg']
eng_list = ['engineSize']
age_list = ['age']
misc_list = ['mileage', 'tax', 'mpg', 'engineSize', 'age']
make_list = ['make_bmw', 'make_ford', 'make_hyundi', 
            'make_merc', 'make_skoda', 'make_toyota', 'make_vauxhall', 'make_vw']
trans_list = ['transmission_Manual', 'transmission_Other', 'transmission_Semi-Auto'] 
fuel_list = ['fuelType_Electric', 'fuelType_Hybrid', 'fuelType_Other', 'fuelType_Petrol'] 

In [ ]:
# Loop over combinations of above lists - 

results = []
for i in [1]:
    for feat_set in itertools.combinations(feat_list, i):
        feature_list = []
        result_list = ""
        for j in range(0,i):
            feature_list += list(feat_set)[j]
        if 'engineSize' in feature_list:
            result_list += 'eng_'
        if 'age' in feature_list:
            result_list += 'age_'
        if 'tax' in feature_list:
            result_list += 'tax_'
        if 'mpg' in feature_list:
            result_list += 'mpg_'
        if 'mileage' in feature_list:
            result_list += 'misc_'
        if 'make_bmw' in feature_list:
            result_list += 'make_'
        if 'model_ 2 Series' in feature_list:
            result_list += 'model_'
        if 'transmission_Manual' in feature_list:
            result_list += 'trans_'
        if 'fuelType_Electric' in feature_list:
            result_list += 'fuel_'
        
        #Split into explanatory and response variables
        X = dum_df[feature_list]
        y = dum_df['price']

        #Split into train and test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=42)

        # Instantiate and fit linear regression model
        lm_model = LinearRegression(normalize=True) 
        lm_model.fit(X_train, y_train) #Fit

        #Predict and score the model
        y_test_preds = lm_model.predict(X_test)
        results.append([result_list,r2_score(y_test, y_test_preds)])
        print("{} - The r-squared score using quantitative variables was {} on {} values.".format(result_list, r2_score(y_test, y_test_preds), len(y_test)))

In [ ]:
#result_df = pd.DataFrame(results, columns = ['features','r2_score'])
#result_df.sort_values(by=['r2_score'], ascending=False).to_csv('results\results.csv')

#result_df.plot.bar(x='features', y='r2_score', rot=0)
#plt.show()